In [1]:
import sys
sys.path.insert(0, '..')
from semantic_mpc_interface import (
    ModelBuilder,
    BuildingMetadataLoader,
    add_points,
    Survey,
    SurveyReader,
    convert_units,
    BrickToGrafana,
    SHACLHandler
)
from buildingmotif.namespaces import BRICK, RDF
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

# Testing the Model Builder

In [2]:
builder = ModelBuilder(
    site_id="test-site",
    ontology = "brick"
)
builder.add_site(
    timezone="America/New_York",
    latitude=40.7128,
    longitude=-74.0060,
    noaa_station="KJFK",
)

/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))


In [3]:
builder.add_zone("zone1")
builder.add_window("window1", "zone1", area_value=10.5, azimuth_value=180, tilt_value=30, unit = 'FT2')
builder.add_thermostat("tstat1", "zone1", stage_count=2, setpoint_deadband=1, tolerance=2, active=True, resolution = 1)
builder.add_hvac("hvac1", "zone1", cooling_capacity=5.0, heating_capacity=4.0, cooling_cop=3.5, heating_cop=3.0)
builder.add_space("space1", "zone1", area_value=50.0)

builder.add_zone("zone2")
builder.add_window("window2", "zone2", area_value=10.5, azimuth_value=180, tilt_value=30, unit = 'FT2')
builder.add_thermostat("tstat2", "zone2", stage_count=2, setpoint_deadband=1, tolerance=2, active=True, resolution =  1)
builder.add_hvac("hvac2", "zone2", cooling_capacity=5.0, heating_capacity=4.0, cooling_cop=3.5, heating_cop=3.0)
builder.add_space("space2", "zone2", area_value=50.0)

zone1
zone2


In [4]:
with open('metadata-survey/s223-filled-out-easy-config/point_list.csv') as f:
    point_list = csv.DictReader(f)
    for row in point_list:
        builder.add_point(row['point_name'], row['point_of'], row['point_template'], row['point_type'], row['ref_name'], row['ref_type'], row['unit'])

In [5]:
builder.save_model("test-brick-model.ttl")

# Testing SHACL Generation and Validation

In [6]:
og = clone.clone_graph(builder.model.graph)

In [7]:
# Create handler
handler = SHACLHandler()

# Generate shapes
handler.generate_shapes()

# Save shapes
handler.save_shapes('shapes.ttl')

# Validate a model
conforms, results_graph, results_text = handler.validate_model(builder.model.graph)

builder.model.graph.serialize('test-brick-model-reasoned.ttl', format = 'ttl')
if not conforms:
    print("Validation failed:")
    print(results_text)

In [8]:
# lots of new inferred information
(builder.graph-og).print()

@prefix ns1: <urn:hpflex/shapes#> .

<urn:hpflex/test-site#hvac1> ns1:air-connects-to <urn:hpflex/test-site#zone1> .

<urn:hpflex/test-site#hvac2> ns1:air-connects-to <urn:hpflex/test-site#zone2> .

<urn:hpflex/test-site#tstat1> ns1:has-location <urn:hpflex/test-site#zone1> ;
    ns1:has-point <urn:hpflex/test-site#tstat1_active>,
        <urn:hpflex/test-site#tstat1_setpoint_deadband>,
        <urn:hpflex/test-site#tstat1_tolerance> .

<urn:hpflex/test-site#tstat2> ns1:has-location <urn:hpflex/test-site#zone2> ;
    ns1:has-point <urn:hpflex/test-site#tstat2_active>,
        <urn:hpflex/test-site#tstat2_setpoint_deadband>,
        <urn:hpflex/test-site#tstat2_tolerance> .

<urn:hpflex/test-site#Active_Cooling_Stages> ns1:has-reference <urn:hpflex/test-site#Active_Cooling_Stages_ref> .

<urn:hpflex/test-site#Active_Heating_Stages> ns1:has-reference <urn:hpflex/test-site#Active_Heating_Stages_ref> .

<urn:hpflex/test-site#Effective_Cooling_Temperature_Setpoint> ns1:has-reference <urn:hp

# Testing get Metadata

In [9]:
site_metadata = BuildingMetadataLoader("test-brick-model.ttl", ontology = 'brick')
site_info = site_metadata.get_site_info()
thermostat_data = site_metadata.get_thermostat_data()

In [10]:
thermostat_data

{'heat_availability': [False, False],
 'cool_availability': [True, True],
 'heat_tolerance': [-2.0, -2.0],
 'cool_tolerance': [2.0, 2.0],
 'setpoint_deadband': [1, 1],
 'active': [True, True],
 'control_group': ['DEPRECATED', 'DEPRECATED'],
 'control_type_list': ['stage', 'stage'],
 'floor_area_list': [50.0, 50.0],
 'window_area_list': [10.5, 10.5],
 'azimuth_list': [180, 180],
 'tilt_list': [30, 30],
 'zone_ids': ['zone1', 'zone2'],
 'hvacs': ['hvac1', 'hvac2'],
 'setpoint_type': ['double', 'double'],
 'fuel_heat_list': ['gas', 'gas'],
 'fuel_cool_list': ['electricity', 'electricity'],
 'cooling_capacity': [5.0, 5.0],
 'heating_capacity': [4.0, 4.0],
 'cooling_cop': [3.5, 3.5],
 'heating_cop': [3.0, 3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [1, 1],
 'temperature_unit': [rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F'),
  rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F')]}

In [11]:
site_metadata.get_thermostat_data(for_zone='zone1')

{'heat_availability': [False],
 'cool_availability': [True],
 'heat_tolerance': [-2.0],
 'cool_tolerance': [2.0],
 'setpoint_deadband': [1],
 'active': [True],
 'control_group': ['DEPRECATED'],
 'control_type_list': ['stage'],
 'floor_area_list': [50.0],
 'window_area_list': [10.5],
 'azimuth_list': [180],
 'tilt_list': [30],
 'zone_ids': ['zone1'],
 'hvacs': ['hvac1'],
 'setpoint_type': ['double'],
 'fuel_heat_list': ['gas'],
 'fuel_cool_list': ['electricity'],
 'cooling_capacity': [5.0],
 'heating_capacity': [4.0],
 'cooling_cop': [3.5],
 'heating_cop': [3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [1],
 'temperature_unit': [rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F')]}

In [12]:
site_metadata.get_complete_output()

{'tz': 'urn:hpflex/test-site#test-site.timezone',
 'latitude': 'urn:hpflex/test-site#test-site.latitude',
 'longitude': 'urn:hpflex/test-site#test-site.longitude',
 'NOAAstation': 'urn:hpflex/test-site#test-site.noaastation',
 'project_id': rdflib.term.URIRef('urn:hpflex/test-site#test-site'),
 'site_id': rdflib.term.URIRef('urn:hpflex/test-site#test-site'),
 'heat_availability': [False, False],
 'cool_availability': [True, True],
 'heat_tolerance': [-2.0, -2.0],
 'cool_tolerance': [2.0, 2.0],
 'setpoint_deadband': [1, 1],
 'active': [True, True],
 'control_group': ['DEPRECATED', 'DEPRECATED'],
 'control_type_list': ['stage', 'stage'],
 'floor_area_list': [50.0, 50.0],
 'window_area_list': [10.5, 10.5],
 'azimuth_list': [180, 180],
 'tilt_list': [30, 30],
 'zone_ids': ['zone1', 'zone2'],
 'hvacs': ['hvac1', 'hvac2'],
 'setpoint_type': ['double', 'double'],
 'fuel_heat_list': ['gas', 'gas'],
 'fuel_cool_list': ['electricity', 'electricity'],
 'cooling_capacity': [5.0, 5.0],
 'heating_ca

# Generating Metadata Survey

In [13]:
survey_gen = Survey(site_id = 'easy-config-test',building_id = 'easy-config-test',
                             hvac_type = 'hp-rtu')
# zone_space_window list for configuration: 3 zones, with 2 spaces and 2 windows, 1 space and 2 windows, 1 space 3 windows, one hvac unit per zone
survey_gen.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)], output_path='./metadata-survey')
# survey_gen.easy_config(zone_space_window_list=[(1,1)], output_path='./metadata-survey')


FileExistsError: Directory 'metadata-survey/easy-config-test/easy-config-test' already exists and is not empty. Please use a different path or clear the directory.

In [ ]:
# Custom config 
# TODO: add underscores to begining of all methods but generate template and easy config
survey_gen = Survey(site_id = 'easy-config-test', building_id = 'easy-config-test',
                             hvac_type = 'vrf')
survey_gen.generate_template(
    hvacs_feed_hvacs = {'HP1': ['FCU1','FCU2','FCU3'],
                        'HP2': ['FCU1', 'FCU2', 'FCU3']},
    hvacs_feed_zones= {'FCU1': ['Zone1'],
                       'FCU2': ['Zone2'],
                       'FCU3': ['Zone3']},
    zones_contain_spaces= {'Zone1': ['Z1S1', 'Z1S2'],
                           'Zone2': ['Z1S2','Z2S3'],
                           'Zone3': ['Z3S1']},
    zones_contain_windows= {'Zone1': ['W1'],
                           'Zone2': ['W2'],
                           'Zone3': ['W3','W4','W5','W6']},
                           output_path='./metadata-survey'
)

FileExistsError: Directory 'metadata-survey/easy-config-test/easy-config-test' already exists and is not empty. Please use a different path or clear the directory.

# Reading Metadata Survey

In [ ]:
generator = SurveyReader("./metadata-survey/filled-out-easy-config-test")
generator.create_model("survey-brick-model.ttl")

/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/buildingmotif/database/table_connection.py:423: SAWarning: Identity map already had an identity for (<class 'buildingmotif.database.tables.DepsAssociation'>, (1,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  self.bm.session.flush()
/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/buildingmotif/database/table_connection.py:423: SAWarning: Identity map already had an identity for (<class 'buildingmotif.database.tables.DepsAssociation'>, (2,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  self.bm.session.flush()
/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/.venv/lib/python3.10/site-packages/buildingmotif/database/table_connection.py:423: SAWarning: Identity map already had an ide

In [ ]:
generator.config['hvacs_feed_zones']

{'hvac_1': ['zone_1']}

# Testing Unit Conversion

In [ ]:
convert_units(10, 'FT', 'M')

3.048

In [ ]:
convert_units(0, 'DEG_C', 'DEG_F')

31.999999999999886

In [ ]:
convert_units(0, 'DEG_C', 'K')

273.15

In [ ]:
site_metadata.convert_model_to_si()
site_metadata.get_thermostat_data(for_zone='zone1')

changing value of  urn:hpflex/test-site#window1_area from http://qudt.org/vocab/unit/FT2 to http://qudt.org/vocab/unit/M2
changing value of  urn:hpflex/test-site#window2_area from http://qudt.org/vocab/unit/FT2 to http://qudt.org/vocab/unit/M2


{'heat_availability': [False],
 'cool_availability': [True],
 'heat_tolerance': [-2.0],
 'cool_tolerance': [2.0],
 'setpoint_deadband': [1],
 'active': [True],
 'control_group': ['DEPRECATED'],
 'control_type_list': ['stage'],
 'floor_area_list': [50.0],
 'window_area_list': [0.9754819200000001],
 'azimuth_list': [180],
 'tilt_list': [30],
 'zone_ids': ['zone1'],
 'hvacs': ['hvac1'],
 'setpoint_type': ['double'],
 'fuel_heat_list': ['gas'],
 'fuel_cool_list': ['electricity'],
 'cooling_capacity': [5.0],
 'heating_capacity': [4.0],
 'cooling_cop': [3.5],
 'heating_cop': [3.0],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [1],
 'temperature_unit': [rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F')]}

In [ ]:
"""
The temperature unit still shows up as DEG_F, because it is at the units for the zone temperature, 
which does not have a value to convert in the model, it's values are in the DB.
it is a reminder to also convert that to C also. We can use the unit_conversion script for any conversions in the db.
"""

"\nThe temperature unit still shows up as DEG_F, because it is at the units for the zone temperature, \nwhich does not have a value to convert in the model, it's values are in the DB.\nit is a reminder to also convert that to C also. We can use the unit_conversion script for any conversions in the db.\n"

# Testing Grafana Dashboarding 

In [ ]:
import yaml 
with open('../development_files/creds.yml') as f:
    config = yaml.safe_load(f)

bg = BrickToGrafana(grafana_server=config['grafana_server'], grafana_api_key = config['grafana_api_key'], datasource=config['datasource'], ttl_path = 'test-brick-model.ttl')

In [ ]:
bg.create_dashboard('AQL from brick')

Dashboard(title='AQL from brick', annotations=Annotations(list=[]), description='Zone-based dashboard generated from Brick model', editable=True, gnetId=None, graphTooltip=0, hideControls=False, id=None, inputs=[], links=[], panels=[], refresh='10s', rows=[], schemaVersion=12, sharedCrosshair=False, style='dark', tags=['brick'], templating=Templating(list=[]), time=Time(start='now-1h', end='now'), timePicker=TimePicker(refreshIntervals=['5s', '10s', '30s', '1m', '5m', '15m', '30m', '1h', '2h', '1d'], timeOptions=['5m', '15m', '1h', '6h', '12h', '24h', '2d', '7d', '30d'], nowDelay=None, hidden=False), timezone='browser', version=0, uid=None)

In [ ]:
# Grafana not set up, upload won't work
# bg.upload_dashboard(message = 'testing upload')